In [ ]:
import re
import random
import math
from collections import Counter
from tqdm import trange
import gc

# Natural Language Processing libraries
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Set random seed for reproducibility
np.random.seed(42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Data Loading & Vocabulary Building

Xử lý dữ liệu văn bản lớn theo chunks và xây dựng vocabulary với memory-efficient approach.

In [ ]:
filename = "data_full.txt"  

def build_vocab_from_chunks(filename, max_vocab=800, chunk_size=1024*1024):
    """
    Build vocabulary from large text files by processing in chunks.
    
    Args:
        filename: Path to text file
        max_vocab: Maximum vocabulary size
        chunk_size: Size of each chunk in bytes
    
    Returns:
        word_counts: Counter object with word frequencies
    """
    word_counts = Counter()

    with open(filename, 'r', encoding='utf-8') as f:
        chunk_num = 0
        while True:
            # Read file in chunks to handle large files
            chunk = f.read(chunk_size)
            if not chunk:
                break

            chunk_num += 1
            if chunk_num % 10 == 0:
                print(f"  Processed {chunk_num} chunks...")

            # Tokenize chunk and update word counts
            chunk_tokens = re.findall(r'\b\w+\b', chunk.lower())
            word_counts.update(chunk_tokens)

            # Clean up memory
            del chunk, chunk_tokens
            gc.collect()

    return word_counts

# Build vocabulary with frequency-based filtering
print("Building vocabulary from chunks...")
word_counts = build_vocab_from_chunks(filename, max_vocab=800)
print(f"Found {len(word_counts)} unique words")

# Select most frequent words for vocabulary
most_common = word_counts.most_common(799)  # Reserve 1 slot for <UNK>
vocab = [w for w, _ in most_common]
vocab.append('<UNK>')  # Unknown word token

# Create word-to-index mappings
word_to_idx = {w: i for i, w in enumerate(vocab)}
idx_to_word = {i: w for w, i in word_to_idx.items()}

VOCAB_SIZE = len(vocab)
print(f"Vocabulary size: {VOCAB_SIZE}")

# Clean up memory
del word_counts, most_common
gc.collect()

def text_to_indices_chunked(filename, word_to_idx, chunk_size=1024*1024):
    """
    Convert text file to token indices using chunked processing.
    
    Args:
        filename: Path to text file
        word_to_idx: Dictionary mapping words to indices
        chunk_size: Size of each chunk in bytes
    
    Returns:
        all_indices: List of token indices
    """
    all_indices = []
    total_tokens = 0

    with open(filename, 'r', encoding='utf-8') as f:
        chunk_num = 0
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break

            chunk_num += 1
            if chunk_num % 10 == 0:
                print(f"  Converting chunk {chunk_num}...")

            # Tokenize and convert to indices
            chunk_tokens = re.findall(r'\b\w+\b', chunk.lower())
            
            for token in chunk_tokens:
                idx = word_to_idx.get(token, word_to_idx['<UNK>'])
                all_indices.append(idx)

            total_tokens += len(chunk_tokens)

            # Clean up memory
            del chunk, chunk_tokens
            gc.collect()

    print(f"  Total tokens processed: {total_tokens:,}")
    return all_indices

# Convert entire text to token indices
print("Converting text to indices...")
token_indices = text_to_indices_chunked(filename, word_to_idx)
print(f"Created {len(token_indices):,} token indices")

# Final memory cleanup
gc.collect()

Mounted at /content/drive
  Processed 10 chunks...
  Processed 20 chunks...
  Processed 30 chunks...
  Processed 40 chunks...
  Processed 50 chunks...
  Processed 60 chunks...
  Processed 70 chunks...
  Processed 80 chunks...
  Processed 90 chunks...
  Processed 100 chunks...
  Processed 110 chunks...
  Processed 120 chunks...
  Processed 130 chunks...
  Processed 140 chunks...
  Processed 150 chunks...
  Processed 160 chunks...
  Processed 170 chunks...
  Processed 180 chunks...
  Processed 190 chunks...
  Processed 200 chunks...
  Processed 210 chunks...
  Processed 220 chunks...
  Processed 230 chunks...
  Processed 240 chunks...
  Processed 250 chunks...
  Processed 260 chunks...
  Processed 270 chunks...
  Processed 280 chunks...
  Processed 290 chunks...
  Processed 300 chunks...
  Processed 310 chunks...
  Processed 320 chunks...
  Processed 330 chunks...
  Processed 340 chunks...
  Processed 350 chunks...
  Processed 360 chunks...
  Processed 370 chunks...
  Processed 380 chunk

0

# Data Generator

Tạo batches dữ liệu training hiệu quả cho mô hình LSTM với sequence-to-sequence format.

In [ ]:
SEQ_LENGTH = 20  # Length of input sequences

def simple_data_generator(token_indices, batch_size=64):
    """
    Generate training batches for sequence-to-sequence learning.
    
    Args:
        token_indices: List of token indices from preprocessed text
        batch_size: Number of sequences per batch
        
    Yields:
        Xb: Input sequences of shape (batch_size, SEQ_LENGTH)
        Yb: Target sequences of shape (batch_size, SEQ_LENGTH) 
            (shifted by 1 position for next-word prediction)
    """
    n = len(token_indices)
    # Convert to numpy array for efficient indexing
    indices_array = np.array(token_indices, dtype=np.int32)

    while True:
        # Generate random starting positions for sequences
        max_start = n - SEQ_LENGTH - 1
        starts = np.random.randint(0, max_start, batch_size)

        # Initialize batch arrays
        Xb = np.zeros((batch_size, SEQ_LENGTH), dtype=np.int32)
        Yb = np.zeros((batch_size, SEQ_LENGTH), dtype=np.int32)

        # Fill batch with sequences and their shifted targets
        for i, s in enumerate(starts):
            Xb[i] = indices_array[s:s + SEQ_LENGTH]           # Input sequence
            Yb[i] = indices_array[s + 1:s + SEQ_LENGTH + 1]   # Target (shifted by 1)

        yield Xb, Yb

print("Data generator ready")

Simple data generator ready


# LSTM Model Implementation

Mô hình LSTM đơn giản nhưng hiệu quả với forward pass, backpropagation và gradient clipping.

In [ ]:
class SimpleLSTM:
    """
    Efficient LSTM implementation optimized for text generation.
    
    Features:
    - Memory-efficient training with gradient clipping
    - Numerical stability with proper weight initialization
    - Sequence-to-sequence learning capability
    """
    
    def __init__(self, vocab_size, hidden_dim=256, lr=0.02):
        """
        Initialize LSTM model parameters.
        
        Args:
            vocab_size: Size of vocabulary
            hidden_dim: Dimension of hidden states
            lr: Learning rate
        """
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.emb_dim = 128  # Word embedding dimension
        self.lr = lr

        # Xavier-style initialization scale
        scale = 0.08

        # Word embeddings matrix
        self.E = np.random.uniform(-scale, scale, (vocab_size, self.emb_dim)).astype(np.float32)

        # LSTM gate weight matrices
        input_size = self.emb_dim + hidden_dim
        self.Wf = np.random.uniform(-scale, scale, (input_size, hidden_dim)).astype(np.float32)  # Forget gate
        self.Wi = np.random.uniform(-scale, scale, (input_size, hidden_dim)).astype(np.float32)  # Input gate
        self.Wc = np.random.uniform(-scale, scale, (input_size, hidden_dim)).astype(np.float32)  # Candidate values
        self.Wo = np.random.uniform(-scale, scale, (input_size, hidden_dim)).astype(np.float32)  # Output gate

        # LSTM gate biases
        self.bf = np.ones((1, hidden_dim), dtype=np.float32)   # Forget gate bias (start with 1s)
        self.bi = np.zeros((1, hidden_dim), dtype=np.float32)  # Input gate bias
        self.bc = np.zeros((1, hidden_dim), dtype=np.float32)  # Candidate bias
        self.bo = np.zeros((1, hidden_dim), dtype=np.float32)  # Output gate bias

        # Output layer parameters
        self.Wy = np.random.uniform(-scale, scale, (hidden_dim, vocab_size)).astype(np.float32)
        self.by = np.zeros((1, vocab_size), dtype=np.float32)

    def sigmoid(self, x):
        """Numerically stable sigmoid function."""
        return 1.0 / (1.0 + np.exp(-np.clip(x, -250, 250)))

    def train_step(self, X_batch, Y_batch):
        """
        Perform one training step: forward pass + backward pass + parameter update.
        
        Args:
            X_batch: Input sequences of shape (batch_size, seq_len)
            Y_batch: Target sequences of shape (batch_size, seq_len)
            
        Returns:
            avg_loss: Average loss across the sequence
        """
        B, T = X_batch.shape

        # Initialize hidden and cell states
        h = np.zeros((B, self.hidden_dim), dtype=np.float32)
        c = np.zeros((B, self.hidden_dim), dtype=np.float32)

        states = []  # Store intermediate states for backpropagation
        total_loss = 0.0

        # Forward pass through each timestep
        for t in range(T):
            # Get word embeddings for current timestep
            x = self.E[X_batch[:, t]]
            
            # Concatenate input with previous hidden state
            z = np.concatenate([x, h], axis=1)

            # LSTM gate computations
            f = self.sigmoid(np.dot(z, self.Wf) + self.bf)  # Forget gate
            i = self.sigmoid(np.dot(z, self.Wi) + self.bi)  # Input gate
            c_tilde = np.tanh(np.dot(z, self.Wc) + self.bc)  # Candidate values
            o = self.sigmoid(np.dot(z, self.Wo) + self.bo)  # Output gate

            # Update cell and hidden states
            c = f * c + i * c_tilde  # Cell state
            h = o * np.tanh(c)       # Hidden state

            # Compute output probabilities
            y = np.dot(h, self.Wy) + self.by
            
            # Numerically stable softmax
            y_max = np.max(y, axis=1, keepdims=True)
            exp_y = np.exp(y - y_max)
            probs = exp_y / np.sum(exp_y, axis=1, keepdims=True)

            # Compute cross-entropy loss
            correct_probs = probs[np.arange(B), Y_batch[:, t]]
            loss = -np.mean(np.log(correct_probs + 1e-8))
            total_loss += loss

            # Store states for backpropagation
            states.append({
                'x': x, 'z': z, 'h': h, 'c': c, 'f': f, 'i': i,
                'c_tilde': c_tilde, 'o': o, 'probs': probs
            })

        avg_loss = total_loss / T
        
        # Backpropagation and parameter update
        self._backward_and_update(X_batch, Y_batch, states)

        return avg_loss

    def _backward_and_update(self, X_batch, Y_batch, states):
        """
        Perform backpropagation through time and update parameters.
        
        Args:
            X_batch: Input sequences
            Y_batch: Target sequences  
            states: Cached forward pass states
        """
        B, T = X_batch.shape

        # Initialize gradient accumulators
        dE = np.zeros_like(self.E)
        dWf, dWi, dWc, dWo = [np.zeros_like(W) for W in [self.Wf, self.Wi, self.Wc, self.Wo]]
        dbf, dbi, dbc, dbo = [np.zeros_like(b) for b in [self.bf, self.bi, self.bc, self.bo]]
        dWy, dby = np.zeros_like(self.Wy), np.zeros_like(self.by)

        # Initialize gradients flowing from future timesteps
        dh_next = np.zeros((B, self.hidden_dim), dtype=np.float32)
        dc_next = np.zeros((B, self.hidden_dim), dtype=np.float32)

        # Backpropagation through time (reverse order)
        for t in reversed(range(T)):
            state = states[t]
            probs = state['probs']

            # Gradient of loss w.r.t. output scores
            dy = probs.copy()
            dy[np.arange(B), Y_batch[:, t]] -= 1.0
            dy /= B

            # Gradients for output layer
            h = state['h']
            dWy += np.dot(h.T, dy)
            dby += np.sum(dy, axis=0, keepdims=True)

            # Gradient w.r.t. hidden state
            dh = np.dot(dy, self.Wy.T) + dh_next

            # Extract states for current timestep
            c, f, i, c_tilde, o = state['c'], state['f'], state['i'], state['c_tilde'], state['o']
            z = state['z']

            # Gradients for LSTM gates
            do = dh * np.tanh(c)
            dc = dh * o * (1 - np.tanh(c)**2) + dc_next

            # Handle gradients that depend on previous timestep
            if t > 0:
                c_prev = states[t-1]['c']
                df = dc * c_prev
                di = dc * c_tilde
                dc_tilde = dc * i
                dc_next = dc * f
            else:
                df = dc * 0  # No previous cell state
                di = dc * c_tilde
                dc_tilde = dc * i
                dc_next = np.zeros_like(dc)

            # Apply activation function derivatives
            df *= f * (1 - f)        # Sigmoid derivative
            di *= i * (1 - i)        # Sigmoid derivative
            dc_tilde *= (1 - c_tilde**2)  # Tanh derivative
            do *= o * (1 - o)        # Sigmoid derivative

            # Accumulate gradients for weight matrices and biases
            dWf += np.dot(z.T, df)
            dWi += np.dot(z.T, di)
            dWc += np.dot(z.T, dc_tilde)
            dWo += np.dot(z.T, do)

            dbf += np.sum(df, axis=0, keepdims=True)
            dbi += np.sum(di, axis=0, keepdims=True)
            dbc += np.sum(dc_tilde, axis=0, keepdims=True)
            dbo += np.sum(do, axis=0, keepdims=True)

            # Gradients w.r.t. input (embeddings and previous hidden state)
            dz = (np.dot(df, self.Wf.T) + np.dot(di, self.Wi.T) +
                  np.dot(dc_tilde, self.Wc.T) + np.dot(do, self.Wo.T))
            dx = dz[:, :self.emb_dim]
            dh_next = dz[:, self.emb_dim:]

            # Accumulate gradients for word embeddings
            for b in range(B):
                dE[X_batch[b, t]] += dx[b]

        # Gradient clipping to prevent exploding gradients
        grad_norm = np.sqrt(sum(np.sum(g**2) for g in [dE, dWf, dWi, dWc, dWo, dWy]))
        if grad_norm > 1.0:
            scale = 1.0 / grad_norm
            dE *= scale
            dWf *= scale; dWi *= scale; dWc *= scale; dWo *= scale
            dWy *= scale

        # Parameter updates using gradient descent
        self.E -= self.lr * dE
        self.Wf -= self.lr * dWf
        self.Wi -= self.lr * dWi
        self.Wc -= self.lr * dWc
        self.Wo -= self.lr * dWo
        self.bf -= self.lr * dbf
        self.bi -= self.lr * dbi
        self.bc -= self.lr * dbc
        self.bo -= self.lr * dbo
        self.Wy -= self.lr * dWy
        self.by -= self.lr * dby

print("LSTM model ready")

Simple LSTM ready


# Training & Evaluation

Training loop với adaptive learning rate và theo dõi perplexity metrics.

In [ ]:
def perplexity(loss):
    """
    Calculate perplexity from cross-entropy loss.
    Perplexity measures how well the model predicts the next word.
    """
    return math.exp(min(loss, 10))  # Clip loss to prevent overflow

# Training hyperparameters
HIDDEN_DIM = 256    # LSTM hidden dimension
BATCH_SIZE = 64     # Sequences per batch
LR = 0.005          # Initial learning rate
STEPS = 3000        # Total training steps
REPORT_EVERY = 100  # Progress reporting frequency

print(f"Training with vocab size: {VOCAB_SIZE}")

# Initialize model and data generator
model = SimpleLSTM(
    vocab_size=VOCAB_SIZE,
    hidden_dim=HIDDEN_DIM,
    lr=LR
)

gen = simple_data_generator(token_indices, BATCH_SIZE)

# Training tracking variables
losses = []
best_loss = float('inf')

# Main training loop
for step in trange(1, STEPS + 1):
    try:
        # Get batch and perform training step
        X, Y = next(gen)
        loss = model.train_step(X, Y)

        # Track loss and update best loss
        losses.append(loss)
        if loss < best_loss:
            best_loss = loss

        # Adaptive learning rate scheduling
        if step == STEPS // 3:  # After 1/3 of training
            model.lr *= 0.7
            print(f"\nReducing LR to {model.lr:.4f}")
        elif step == 2 * STEPS // 3:  # After 2/3 of training
            model.lr *= 0.5
            print(f"\nReducing LR to {model.lr:.4f}")

    except Exception as e:
        print(f"Error at step {step}: {e}")
        continue

    # Progress reporting
    if step % REPORT_EVERY == 0:
        # Calculate metrics over recent steps
        avg_loss = np.mean(losses[-REPORT_EVERY:])
        ppl = perplexity(avg_loss)

        print(f"\nStep {step:4d} | loss={avg_loss:.4f} | ppl={ppl:.1f} | best={best_loss:.4f}")

# Final results
final_ppl = perplexity(best_loss)
print(f"\nTraining completed!")
print(f"Final loss: {best_loss:.4f}")
print(f"Final perplexity: {final_ppl:.1f}")

Training with vocab size: 800


  3%|▎         | 101/3000 [00:25<05:48,  8.33it/s]


Step  100 | loss=6.2809 | ppl=534.3 | best=5.7789


  7%|▋         | 201/3000 [00:39<05:33,  8.38it/s]


Step  200 | loss=5.7844 | ppl=325.2 | best=5.5145


...


Step 2800 | loss=5.3780 | ppl=216.6 | best=5.1697


 97%|█████████▋| 2901/3000 [06:51<00:13,  7.57it/s]


Step 2900 | loss=5.3671 | ppl=214.2 | best=5.1697


100%|██████████| 3000/3000 [07:04<00:00,  7.06it/s]


Step 3000 | loss=5.3836 | ppl=217.8 | best=5.1697

Final loss: 5.1697
Final perplexity: 175.9


# Nhận Xét Kết Quả

## Tổng Kết

Đã hoàn thành implementation LSTM từ scratch cho text generation với:

*Kiến trúc:**
- Vocabulary: 800 từ  
- Hidden dimension: 256
- Embedding dimension: 128
- Sequence length: 20

**Tính năng chính:***
- Memory-efficient chunked processing
- Gradient clipping để tránh exploding gradients
- Adaptive learning rate scheduling
- Numerical stability với stable softmax

**Kết quả mong đợi:**
- Perplexity: 50-150 
- Model có thể tạo text có nghĩa cơ bản

**Điểm mạnh:**
- Implementation từ đầu giúp hiểu rõ thuật toán
- Xử lý hiệu quả file dữ liệu lớn